代码示例说明:

这个代码示例演示了如何使用 LangChain 构建一个基于 PDF 文档的语义搜索引擎。它包含了文档加载、分割、嵌入生成、向量存储和检索等关键步骤。

文档加载: 使用 PyPDFLoader 从 PDF 文件加载文档，每页一个文档。
文档分割: 使用 RecursiveCharacterTextSplitter 将文档分割成更小的块，以便更好地进行语义搜索。
嵌入生成: 使用 OpenAIEmbeddings 将文本块转换为嵌入向量，这些向量捕捉了文本的语义信息。
向量存储: 使用 Chroma 存储嵌入向量，以便进行高效的相似性搜索。
向量检索: 使用向量存储进行相似性搜索，找到与查询最相关的文档块。
检索器: 创建可重用的检索器对象，方便进行批量查询。
解决的问题:

传统的关键词搜索只能找到包含特定关键词的文档，而语义搜索则可以找到含义与查询相关的文档，即使文档中没有包含查询的关键词。这个代码示例展示了如何使用 LangChain 构建一个语义搜索引擎，从而更准确地找到用户感兴趣的信息。

达到的效果:

通过这个代码示例，你可以：

加载和处理 PDF 文档。
将文本转换为嵌入向量，捕捉语义信息。
使用向量存储进行高效的相似性搜索。
构建一个可重用的检索器对象。
这个示例可以帮助你理解 LangChain 的核心概念，并为构建更复杂的自然语言处理应用打下基础。  代码中添加了详细的中文注释，解释了每个步骤的作用和目的，方便理解和学习。



In [ ]:
!pip install langchain \
            langchain-community \
            langchain-openai \
            langchain-text-splitters \
            pypdf \
            chromadb

In [ ]:
# 导入必要的库
import getpass
import os
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 设置 LangSmith 追踪，用于调试和监控 LangChain 应用
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

# 1. 加载文档
# 使用 PyPDFLoader 从 PDF 文件加载文档
file_path = "../example_data/nke-10k-2023.pdf"  # PDF 文件路径
loader = PyPDFLoader(file_path)
docs = loader.load()

print(f"加载了 {len(docs)} 个文档（每页一个文档）")

# 打印第一个文档的内容和元数据
print(f"第一个文档的内容（前200个字符）：\n{docs[0].page_content[:200]}\n")
print(f"第一个文档的元数据：\n{docs[0].metadata}")


# 2. 分割文档
# 使用 RecursiveCharacterTextSplitter 将文档分割成更小的块
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(f"将文档分割成 {len(all_splits)} 个块")

# 3. 生成嵌入
# 使用 OpenAIEmbeddings 创建嵌入模型
embeddings = OpenAIEmbeddings()

# 示例：生成两个文本块的嵌入向量并比较长度
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)  # 确保向量长度一致
print(f"生成的向量长度为 {len(vector_1)}\n")
print(f"第一个向量的前10个元素：\n{vector_1[:10]}")

# 4. 创建向量存储
# 使用 Chroma 创建向量存储
vector_store = Chroma(embedding_function=embeddings)

# 将分割后的文档块添加到向量存储
ids = vector_store.add_documents(documents=all_splits)

print(f"将 {len(all_splits)} 个文档块添加到向量存储")


# 5. 使用向量存储进行搜索
# 示例：使用相似性搜索查询文档
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(f"相似性搜索结果：\n{results[0]}")

# 示例：异步查询
async def async_search():
    results = await vector_store.asimilarity_search("When was Nike incorporated?")
    print(f"异步搜索结果：\n{results[0]}")

#import asyncio
#asyncio.run(async_search())


# 示例：返回相似性得分
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"相似性得分：{score}\n")
print(f"带得分的搜索结果：\n{doc}")

# 示例：使用嵌入向量搜索
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")
results = vector_store.similarity_search_by_vector(embedding)
print(f"基于向量的搜索结果：\n{results[0]}")


# 6. 创建检索器
# 自定义检索器示例
@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

# 批量检索示例
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

# 使用 as_retriever 创建检索器
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

# 批量检索示例
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

print("检索器示例完成")